<a href="https://colab.research.google.com/github/owgee/supply-chain-analytics/blob/main/Tailored_Sourcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Products are sourced independently;

In [1]:
import math

products = {
    'Product 1': {'Demand': 1000, 'Ordering_cost': 10, 'Unit_cost': 50, 'Holding_cost_rate': 0.2},
    'Product 2': {'Demand': 300, 'Ordering_cost': 20, 'Unit_cost': 60, 'Holding_cost_rate': 0.2},
    'Product 3': {'Demand': 100, 'Ordering_cost': 25, 'Unit_cost': 30, 'Holding_cost_rate': 0.2},
    'Product 4': {'Demand': 50, 'Ordering_cost': 25, 'Unit_cost': 30, 'Holding_cost_rate': 0.2},
}
common_cost = 100;

# EOQ Calculation
def calculate_eoq(product_info):
    D = product_info['Demand']
    S = product_info['Ordering_cost'] + common_cost
    H = product_info['Unit_cost'] * product_info['Holding_cost_rate']
    return math.sqrt((2 * D * S) / H)

# Total Cost Calculation for EOQ
def calculate_total_cost(product_info, eoq):
    D = product_info['Demand']
    S = product_info['Ordering_cost'] + common_cost
    H = product_info['Unit_cost'] * product_info['Holding_cost_rate']
    total_ordering_cost = D * S / eoq
    total_holding_cost = eoq * H / 2
    return total_ordering_cost + total_holding_cost

# Calculate EOQ and total cost for each product
total_annual_cost=0
for product in products:
    eoq = calculate_eoq(products[product])
    total_cost = calculate_total_cost(products[product], eoq)
    total_annual_cost += total_cost
    print(f"{product} - EOQ: {eoq:.2f}, Total Cost: {total_cost:.2f}")
print(f"Total Annual Cost: {total_annual_cost:.2f}")

Product 1 - EOQ: 148.32, Total Cost: 1483.24
Product 2 - EOQ: 77.46, Total Cost: 929.52
Product 3 - EOQ: 64.55, Total Cost: 387.30
Product 4 - EOQ: 45.64, Total Cost: 273.86
Total Annual Cost: 3073.92


### 2. All four products are sourced with the same frequency;

In [2]:
import numpy as np

# Given data from the problem statement
demand = [1000, 300, 100, 50]
specific_ordering_costs = [10, 20, 25, 25]
unit_cost = [50, 60, 30, 30]
holding_cost_rate = 0.2
common_ordering_cost = 100   # Provided common ordering cost

# Calculate the total demand times holding cost times unit cost
total_d_h_c = sum(d * uc * holding_cost_rate for d, uc in zip(demand, unit_cost))

# Calculate the total specific ordering costs
total_specific_ordering_cost = sum(specific_ordering_costs)
S = common_ordering_cost + total_specific_ordering_cost

# Calculate n*, the order frequency using the provided common ordering cost and total specific ordering costs
n_star = np.sqrt((total_d_h_c) / (2 * S))

annual_total_cost = n_star * 2 * S
print(f"The annual total cost is: {annual_total_cost:.2f}")

The annual total cost is: 2284.73


### 3. Order frequencies are determined according to the tailored aggregation strategy.

In [6]:
adjusted_order_sizes = [math.sqrt((2 * demand[0] * 100) / (unit_cost[0] * holding_cost_rate))]
adjusted_order_sizes.extend([math.sqrt((2 * d * soc) / (uc * holding_cost_rate)) for d, soc, uc in zip(demand[1:], specific_ordering_costs[1:], unit_cost[1:])])

adjusted_frequencies = [d / aos for d, aos in zip(demand, adjusted_order_sizes)]

multipliers = [round(adjusted_frequencies[0] / freq) for freq in adjusted_frequencies]

numerator = sum(holding_cost_rate * unit_cost[i] * demand[i] * multipliers[i] for i in range(len(demand)))
denominator = 2 * (common_ordering_cost + sum(specific_ordering_costs[i] / multipliers[i] for i in range(len(demand))))

# new order frequency for the most frequently ordered product
n = math.sqrt(numerator / denominator)
optimal_frequencies = [(n / mult) for mult in multipliers]
optimal_order_sizes = [d / f for d, f in zip(demand, optimal_frequencies)]
new_holding_costs = [(nos / 2) * holding_cost_rate * uc for nos, uc in zip(optimal_order_sizes, unit_cost)]


print(f"Multipliers: {multipliers}")
print(f"Baseline frequency: {n}")
print(f"Adjusted order sizes for each product: {adjusted_order_sizes}")
print(f"Adjusted frequencies: {adjusted_frequencies}")
print(f"Optimal frequencies: {optimal_frequencies}")
print(f"Optimal order sizes: {optimal_order_sizes}")
print(f"New holding costs for each product: {new_holding_costs}")
# at optimal, holding cost = ordering cost
print(f"Total annual costs: {round(sum(new_holding_costs*2),2)}")

Multipliers: [1, 1, 2, 3]
Baseline frequency: 7.21416654195255
Adjusted order sizes for each product: [141.4213562373095, 31.622776601683793, 28.867513459481287, 20.412414523193153]
Adjusted frequencies: [7.071067811865475, 9.486832980505138, 3.464101615137755, 2.449489742783178]
Optimal frequencies: [7.21416654195255, 7.21416654195255, 3.607083270976275, 2.40472218065085]
Optimal order sizes: [138.61615117764455, 41.58484535329337, 27.723230235528913, 20.792422676646687]
New holding costs for each product: [693.0807558882228, 249.50907211976022, 83.16969070658674, 62.37726802994007]
Total annual costs: 2176.27
